In [1]:
import HMM
import hmmlearn as hmm
import numpy as np

In [2]:
data = HMM.load_object('small_data.pkl')
obs = np.array(data['data']['cleaned_data'].values)
labels = data['data']['target'].values
symbols = np.arange(len(data['dictionary']))
train = int(2*len(obs)//3)

# setup the hmm params
model = HMM.SentimentModel(symbols)
model.fit(obs[train:], labels)
print(model.predict(obs[:train]))


Fitting a model with 32667 free scalar parameters with only 16 data points will result in a degenerate solution.
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Fitting a model with 32667 free scalar parameters with only 5 data points will result in a degenerate solution.
Even though the 'emissionprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'e'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Fitting a model with 32667 free scalar parameters with only 3 data points will result in a degenerate solution.
Even though the 'emissionpr

None
